In [1]:
import time
import random
import os

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

import undetected_chromedriver as uc


KeyboardInterrupt



In [ ]:
def wait_for_page_to_load(driver, wait):
    title = driver.title
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
    except:
        print(f"Page Loading failed: {title}")
    else:
        print(f"Page Loading successful: {title}")

In [ ]:
def grab_data(driver):    
    data = []
    rows = driver.find_elements(By.CLASS_NAME, 'tupleNew__contentWrap') 
    for row in rows:
        #property name
        try:
            name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
        except:
            name = np.nan

        try:
            location = row.find_element(By.CLASS_NAME, 'tupleNew__propType').text
        except:
            location = np.nan

        #property price
        try:
            price_element = row.find_elements(By.CLASS_NAME, 'tupleNew__priceValWrap')
        except:
            price = np.nan
        else:
            price = price_element[0].text

        #Area Type
        try:
            area_type = row.find_element(By.CLASS_NAME, 'tupleNew__areaType').text
        except:
            area_type = np.nan

        #area
        try:
            area_bhk = row.find_elements(By.CLASS_NAME, 'tupleNew__totolAreaWrap')
        except:
            area, bhk = np.nan, np.nan
        else:
            area = area_bhk[0].text
            bhk = area_bhk[1].text

        property = {
            'name' : name,
            'location' : location,
            'price' : price,
            'area_type' : area_type,
            'area' : area,
            'bhk' : bhk
        }

        data.append(property)
        
    df_page = pd.DataFrame(data)

    return df_page

In [ ]:
def set_filters(driver):
    #scrolling page for elements to load
    driver.execute_script("window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' });")
    time.sleep(2)
    driver.execute_script("window.scrollTo({ top: -document.body.scrollHeight, behavior: 'smooth' });")
    time.sleep(3)
    
    
    #1)ready_to_move
    element_path = '/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[5]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #2)verified
    element_path = '/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[3]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #3)residential_apartment
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[5]/div/div/div[2]/div[1]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #4)resale
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[12]/div/div/div'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)
    
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[12]/div/div/div[2]/div[1]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #5)with_photos
    element_path = '/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[6]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #6)2bhk
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[6]/div/div/div[2]/div[2]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #7)3bhk
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[6]/div/div/div[2]/div[3]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #8)4bhk
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[6]/div/div/div[2]/div[4]/span[2]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)
    
    #9)max_budget_dropdown
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[4]/div/div[2]/div/div[3]/div[1]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)
    
    #10)max_budget
    element_path = '/html/body/div[1]/div/div/div[4]/div[2]/div/div[4]/div/div[2]/div/div[3]/div[2]/div/div[2]/div/ul/li[71]'
    element = driver.find_element(By.XPATH, element_path)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)

    return driver

In [ ]:
def is_captcha_present(driver):
    # Check for common captcha keywords in page title or body
    if "captcha" in driver.page_source.lower() or "access denied" in driver.title.lower():
        return True
    return False

In [ ]:
city = "Gurgaon"

chrome_options = Options()
chrome_options.add_argument("--disable-http2")
#chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
)
#chrome_options.add_argument("--headless")
chrome_options.page_load_strategy = 'eager'



base_url = "https://www.99acres.com/search/property/buy/gurgaon?city=8&preference=S&area_unit=1&res_com=R"

df_start_page = pd.read_csv("page.csv")
start_page = df_start_page.iloc[0,0]

for page in range(start_page,251):
    
    df = pd.read_csv("99_acres_scraped.csv")
    driver = uc.Chrome(options=chrome_options)
    driver.maximize_window()
    wait = WebDriverWait(driver, 10)
    
    if page <= 5:
        url = base_url
    else:
        url = f"{base_url}&page={page}"
    
    driver.get(url)
    wait_for_page_to_load(driver, wait)
    time.sleep(3)

    if is_captcha_present(driver):
        print("CAPTCHA DETECTED! SCRIPT PAUSED.")
        print("Please manually solve the CAPTCHA in the browser window.")
        input("Press Enter here in the console once you have solved it...")
        print("Resuming scraping...")
    
    scroll_height = random.randint(300, 1000)
    #driver.execute_script(f"window.scrollTo({0, scroll_height});")
    driver.execute_script(f"""window.scrollTo({{top: {scroll_height}, behavior: 'smooth'}});""")
    time.sleep(random.uniform(3, 7))
    
    driver = set_filters(driver)
    
    #scrolling loop
    prev_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script("window.scrollTo({ top: document.body.scrollHeight, behavior: 'smooth' });")
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')
        if prev_height == new_height:
            break
        prev_height = new_height
    
    #scraping the data
    print(f"Page {page} : Scraping data...")
    df_page = grab_data(driver)
    df = pd.concat([df,df_page], ignore_index=True)

    sleep_time = random.uniform(5, 12)
    print(f"Sleeping for {sleep_time:.2f} seconds...")
    time.sleep(sleep_time)
    df.to_csv("99_acres_scraped.csv", index=False, encoding="utf-8")

    start_page += 1
    df_page = pd.DataFrame([start_page])
    df_page.to_csv("page.csv", index=False, encoding="utf-8")
    
    driver.quit()
    time.sleep(3)